In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(
    'Streaming Word Count'
).master('local[3]').config(
    'spark.streaming.stopGracefullyOnShutdown', 'true'
).config(
    'spark.sql.shuffle.partitions', 3
).getOrCreate()

23/08/06 12:05:34 WARN Utils: Your hostname, OMEN resolves to a loopback address: 127.0.1.1; using 172.19.181.52 instead (on interface eth0)
23/08/06 12:05:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/06 12:05:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
from pyspark.sql import functions as F

In [5]:
lines_df = spark.readStream.format('socket').option(
    'host', 'localhost'
).option(
    'port', '9999'
).load()

23/08/06 12:06:12 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [6]:
lines_df.printSchema()

root
 |-- value: string (nullable = true)



In [7]:
words_df = lines_df.select(F.expr('explode(split(value, " "))').alias('word'))
counts_df = words_df.groupBy('word').count()

In [9]:
word_count_query = counts_df.writeStream.format(
    'console'
).option('checkpointLocation', 'chk-point-dir').outputMode('complete').start()

word_count_query.awaitTermination()

23/08/06 12:07:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
| Hello|    1|
|There!|    1|
+------+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
| Hello|    1|
|There!|    2|
|   Hey|    1|
+------+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
| World|    1|
| Hello|    2|
|There!|    2|
|   Hey|    1|
+------+-----+



23/08/06 12:09:34 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 